In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = np.load("/Users/dongtianchi/Documents/GIT/SpectralReconstruction/ComputationalSpectrometers/Deeplearning/DATA/data_daylight.npy", allow_pickle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data.T)

In [3]:
class SSDAE(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SSDAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

input_size = 400
hidden_size1 = 128
hidden_size2 = 64
ssdae1 = SSDAE(input_size, hidden_size1)
ssdae2 = SSDAE(hidden_size1, hidden_size2)


In [4]:
#dataloader 构建 + 张量转换
batch_size = 16
data_normalized = np.asarray(data_normalized, dtype=np.float32)
train_loader = DataLoader(TensorDataset(torch.tensor(data_normalized)), batch_size=batch_size, shuffle=True)

In [5]:
loss_function = nn.MSELoss()
optimizer1 = torch.optim.Adam(ssdae1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(ssdae2.parameters(), lr=0.001)

In [7]:
num_epochs = 50
for epoch in range(num_epochs):
    for data in train_loader:
        x = data[0]
        x = x.view(x.size(0), -1)
        noise_factor = 0.5
        
        # 添加噪声
        noisy_x = x + noise_factor * torch.randn(x.shape)

        # 自编码器 1
        output1 = ssdae1.encoder(noisy_x)
        reconstructed1 = ssdae1.decoder(output1)
        loss1 = loss_function(reconstructed1, x)

        optimizer1.zero_grad()
        loss1.backward()
        optimizer1.step()

        # 自编码器 2
        output2 = ssdae2.encoder(output1.detach())  # 使用自编码器 1 的编码器输出作为自编码器 2 的输入
        reconstructed2 = ssdae2.decoder(output2)
        loss2 = loss_function(reconstructed2, output1.detach())

        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss1: {loss1.item():.4f}, Loss2: {loss2.item():.4f}')


Epoch [1/50], Loss1: 0.0017, Loss2: 0.1404
Epoch [2/50], Loss1: 0.0013, Loss2: 0.1947
Epoch [3/50], Loss1: 0.0022, Loss2: 0.1429
Epoch [4/50], Loss1: 0.0022, Loss2: 0.2248
Epoch [5/50], Loss1: 0.0014, Loss2: 0.1715
Epoch [6/50], Loss1: 0.0019, Loss2: 0.1597
Epoch [7/50], Loss1: 0.0016, Loss2: 0.2079
Epoch [8/50], Loss1: 0.0013, Loss2: 0.2556
Epoch [9/50], Loss1: 0.0009, Loss2: 0.2105
Epoch [10/50], Loss1: 0.0007, Loss2: 0.2184
Epoch [11/50], Loss1: 0.0015, Loss2: 0.2793
Epoch [12/50], Loss1: 0.0020, Loss2: 0.1705
Epoch [13/50], Loss1: 0.0011, Loss2: 0.2750
Epoch [14/50], Loss1: 0.0010, Loss2: 0.1655
Epoch [15/50], Loss1: 0.0016, Loss2: 0.2625
Epoch [16/50], Loss1: 0.0009, Loss2: 0.2242
Epoch [17/50], Loss1: 0.0006, Loss2: 0.1849
Epoch [18/50], Loss1: 0.0012, Loss2: 0.1592
Epoch [19/50], Loss1: 0.0008, Loss2: 0.2422
Epoch [20/50], Loss1: 0.0008, Loss2: 0.2666
Epoch [21/50], Loss1: 0.0007, Loss2: 0.1447
Epoch [22/50], Loss1: 0.0013, Loss2: 0.1368
Epoch [23/50], Loss1: 0.0015, Loss2: 0.19

In [8]:
for epoch in range(num_epochs):
    ssdae1.train()
    for batch in train_loader:
        x = batch[0]
        noise_factor = 0.5
        noisy_x = x + noise_factor * torch.randn_like(x)
        noisy_x = torch.clamp(noisy_x, 0., 1.)

        optimizer1.zero_grad()
        outputs = ssdae1(noisy_x)
        loss = loss_function(outputs, x)
        loss.backward()
        optimizer1.step()

    ssdae1.eval()
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}")

Epoch [1/50], Train Loss: 0.0016
Epoch [2/50], Train Loss: 0.0009
Epoch [3/50], Train Loss: 0.0016
Epoch [4/50], Train Loss: 0.0020
Epoch [5/50], Train Loss: 0.0015
Epoch [6/50], Train Loss: 0.0021
Epoch [7/50], Train Loss: 0.0012
Epoch [8/50], Train Loss: 0.0011
Epoch [9/50], Train Loss: 0.0029
Epoch [10/50], Train Loss: 0.0013
Epoch [11/50], Train Loss: 0.0025
Epoch [12/50], Train Loss: 0.0014
Epoch [13/50], Train Loss: 0.0019
Epoch [14/50], Train Loss: 0.0010
Epoch [15/50], Train Loss: 0.0022
Epoch [16/50], Train Loss: 0.0016
Epoch [17/50], Train Loss: 0.0022
Epoch [18/50], Train Loss: 0.0013
Epoch [19/50], Train Loss: 0.0014
Epoch [20/50], Train Loss: 0.0008
Epoch [21/50], Train Loss: 0.0016
Epoch [22/50], Train Loss: 0.0015
Epoch [23/50], Train Loss: 0.0012
Epoch [24/50], Train Loss: 0.0014
Epoch [25/50], Train Loss: 0.0020
Epoch [26/50], Train Loss: 0.0014
Epoch [27/50], Train Loss: 0.0015
Epoch [28/50], Train Loss: 0.0005
Epoch [29/50], Train Loss: 0.0013
Epoch [30/50], Train Lo